# Generating Samples
----
### Introduction
Here we want to generate binary samples of length $v$ using Gibbs sampling based on a prior distribution based on a $v \times v$ matrix $W$ initialized from a known distribution. Using the samples generated, we would eventually want to reproduce/learn the matrix $W$ from the samples using minimum probability flow.

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import time

% matplotlib inline

We shall consider a network with $v$ vertices where each vertex is binary. For a network with $v$ vertices, there are $2^v$ possibles binary states. We initialize an initial state of the network using the Bernoulli distribution with success probability $p$.

In [2]:
# Number of neurons:
v = 16
# Success probability:
p = 0.5

In [3]:
np.random.seed(0)
initialState = np.random.binomial(1, p, v)
print ('Initial states: ',initialState)

Initial states:  [1 1 1 1 0 1 0 1 1 0 1 1 1 1 0 0]


We now initialize a $v$ by $v$ matrix $W$ with each entry drawn from a standard normal distribution, $N(0,1)$. For each entry in the matrix, $W_{ij}$ denotes the parameter/weight associated with the connection from unit $i$ to $j$ (can we think of it as the conditional weight of $v^{(t+1)}_i=1$ given $v^{(t)}_j=1$?). Here we save the matrix $W$ so we can verify the learning by MPF. 

(Personal notes: Later, we will learn that initializing the matrix $W$ with zero diagonals will make it easier in the generation of samples.)

In [4]:
np.random.seed(0)
# Get a symmetric matrix with diagonal all zeros
W = np.random.normal(0, 1, (v,v))
W = 0.5 * (W + np.transpose(W))

# To save and load W matrix
np.save('W.dat', W)
# W = np.load('W.dat')
print (W)

[[ 1.76405235  0.94711814  0.04547612  0.31349768  1.02249207 -1.07121386
   0.48029422 -0.50629145  0.28453795  1.39687164 -0.17699444  0.32543192
   0.36087745  0.20079981 -0.54846689 -0.15188135]
 [ 0.94711814 -0.20515826 -0.83386438 -0.53341801 -1.47738538  0.77722254
   1.32515335  0.58394997  1.33860823 -0.25494309  0.98764529 -0.43836681
   0.18234286  0.68560419  0.10910471 -0.00955465]
 [ 0.04547612 -0.83386438 -0.34791215 -0.3695588  -0.19995383  0.83402114
  -0.13020736 -0.28515306 -0.90923452 -1.16642008 -0.37842469  0.74762145
   0.00681224  0.2360523  -0.54814441  0.32230489]
 [ 0.31349768 -0.53341801 -0.3695588   0.3869025  -0.02401144 -1.35843794
   0.18690357  0.61539413  0.30288321  0.70974409 -0.27338543 -0.172631
  -0.42200339 -0.02116747 -0.29048262 -1.01203674]
 [ 1.02249207 -1.47738538 -0.19995383 -0.02401144 -0.90729836  0.77009879
   1.30612063  0.53811744  0.23253401 -1.27536662 -0.41154694 -0.519402
  -0.0495033   0.10373915 -0.4544343  -0.12642381]
 [-1.0712

### How to do Gibbs Sampling
The reason for doing Gibbs sampling is to generate samples $\mathcal{S}$ from known parameters $W$ and then use MPF to learn the parameters $W$ using $\mathcal{S}$. To sample from this multivariate distribution, we start with an initial state obtained from a prior belief, following which sampling from the conditional distribution is done to get a new state of a **vertex**. Thus if we were to sample each vertex sequentially, a network with $v$ vertices would require sampling from (different) conditional distributions $v$ times for a new state of the network to be obtained.

#### Algorithm: Gibbs sampler (cycle)
1. Initialize $\mathbf{x^{(0)}}=(x_1^{(0)},\ldots,x_v^{(0)})$ base on some prior belief.
2. For $i = 1,2, \ldots$
    - sample $X_1^{(i)}\sim \mathbb{P}(X_1^{(i)}=x_1^{(i)}\mid X_2=x_2^{(i-1)},\ldots,X_v=x_v^{(i-1)})$
    - sample $X_2^{(i)}\sim \mathbb{P}(X_2^{(i)}=x_2^{(i)}\mid X_1=x_1^{(i)},X_3=x_3^{(i-1)}\ldots,X_v=x_v^{(i-1)})$
    - in general, sample $X_j^{(i)}\sim \mathbb{P}(X_{j}^{(i)}=x_{j}^{(i)}\mid X_1=x_1^{(i)},\ldots, X_{j-1}=x_{j-1}^{(i)},X_{j+1}=x_{j+1}^{(i-1)},\ldots,X_v=x_v^{(i-1)})$ for $j=1,2, \ldots v$, which then generates a new state for the network.

There is also another variation of the Gibbs sampler called the **random scan** where the update of the state of the vertex is not in a cycle but done in a random manner. Below are some functions that are defined for the implementation of the Gibbs sampler.

In [5]:
def sigmoid(x):
    """
    Takes in a vector x and returns its sigmoid activation.
    Input:
    - x: a numpy array
    """
    return 1/(1 + np.exp(-x))


def single_unit_update(initialState, W, v):
    """
    Returns the new states and the state of the vth vertex that has been updated conditioned on the other units
    Input:
    - initialState: a numpy array of binary values denoting the initial state of the nodes.
    - W: a 2d numpy array of values that the prior distribution is based from. 
    - v: (int) the state of the vertex to be updated.
    """
    stateSize = initialState.shape[0]
    newState = initialState
#     Here we see that to update a single vertex state we only use the weights Wij for i not 
#     equal to j and hence the reason to set the diagonals to be zero earlier. But since 
#     we did not we have to kill off the diagonals of W here.
    prob = sigmoid((W - (W * np.eye(stateSize))).dot(initialState))
    newState[v] = np.random.binomial(1, prob[v], 1)
#     print (initialState[n], newState[n])
    return newState, newState[v]


def gibbs_sample(initialState, W):
    """
    Returns the new state of the network after updating all v units systematically, given an initialized state 
    of the network and weight matrix W.
    Input:
    - initialState: a numpy array of binary values denoting the initial state of the nodes.
    - W: a 2d numpy array.
    """
#     print ('initialState:', initialState)
    stateSize = initialState.shape
    newState = np.zeros(stateSize)
    for i in range(stateSize[0]):
#         print ('Changing the state for unit %d...'% i)
        initialState, vertexState = single_unit_update(initialState, W, i)
#         print ('Old unit state is %d, new unit state is %d'% (initialState[i], unitState))
        newState[i] = vertexState  
#     print ('newState:', newState)
    return newState    


def multi_gibbs_sample(initialState, W, n):
    """
    Performs Gibbs sampling n times with a given initial state and weight matrix W
    and stores each sample as a row.
    Input:
    - initialState: a numpy array of binary values denoting the initial state of the nodes.
    - W: a 2d numpy array. 
    - n: (int) number of samples to be drawn.
    """
    stateSize = initialState.shape[0]
    sample = np.zeros((n, stateSize))
    for i in range(n):
        sample[i, :] = gibbs_sample(initialState, W)
    return sample    

def rand_gibbs_sample(initialState, W, n):
    """
    Does a random scan Gibbs sampling n times with a given initial state and weight matrix W.
    - initialState: a numpy array of binary values denoting the initial state of the nodes.
    - W: a 2d numpy array. 
    - n: (int) number of samples to be drawn.
    """
#     v = W.shape[0]
#     sample = np.zeros((n, initialState.shape[0]))
    for i in range(n):
        s = np.random.randint(0, v)
        initialState, vertexState = single_unit_update(initialState, W, s)
#         sample[i, :] = gibbs_sample(initialState, W)
    return initialState

To make ensure that the sample that we obtain are independent and identically distributed, we do a **burn-in** of $10000\times v$ iterations so that the samples obtained follow the distribution of the weight matrix, following which we pick a sample for every $1000 \times v$ iterations, which is called **mixing-in**.

### Sampling using random scan Gibbs sampler:

In [6]:
# Burn-in
burnin_state = rand_gibbs_sample(initialState, W, 10000 * v)
print ('Burn-in state: ', burnin_state)

Burn-in state:  [1 1 1 1 0 0 1 1 1 1 0 0 0 1 0 0]


Mixing in still halfway done...

In [9]:
# Mixing-in
def mixin_gibbs_sample(initialState, W, n, m):
    """
    Does a random scan Gibbs sampling n * m times with a given initial state and weight matrix W and 
    stores a sample every m iterations.
    - initialState: a numpy array of binary values denoting the initial state of the nodes.
    - W: a 2d numpy array. 
    - n: (int) number of samples to be drawn.
    """
    tic = time.time()
    v = W.shape[0]
    sample = np.zeros((n, initialState.shape[0]))
    for i in range(n):
#         if i % 5000 == 0:
#             print ('Creating %d sample' % (i))
        sample[i, :] = rand_gibbs_sample(initialState, W, m)
#         sample[i, :] = 
#         if i % m == 0:
            
#         s = np.random.randint(0, v)
#         initialState, vertexState = single_unit_update(initialState, W, s)
#         if i % m == 0:
#             sample[i, :] = gibbs_sample(initialState, W)
    toc = time.time()
    print ('Time taken to create %d samples is %f minutes' % (n, (toc - tic)/60))
    return sample

In [12]:
sample = mixin_gibbs_sample(burnin_state, W, 50000, 100)

Time taken to create 50000 samples is -3.001808
